In [ ]:
import numpy as np
import sklearn
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as pyplot
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split

## Importing dataset

In [ ]:
data, target = load_wine(return_X_y=True)

Data and target examples

In [ ]:
print("Data ",data[130],"\ntarget ",target[130])

Getting indexes in order to plot with different colors

In [ ]:
print(len(target[target==0]))
print(len(target[target==1]))
print(len(target[target==2]))

In [ ]:
pyplot.plot(data[:, :2][:59], 'bo', data[:, :2][59:59+71], 'go', data[:, :2][59+71:59+71+48], 'ro')

### Splitting data in Train (50%), Validation (20%) and Test (30%) 

In [ ]:
Data_train, Data_test, Target_train, Target_test = train_test_split(data, target, test_size=0.30, random_state=42)
Data_train, Data_validation, Target_train, Target_validation = train_test_split(Data_train, Target_train, test_size=0.20, random_state=41)

## kNN

In [ ]:
#plotting method
step_size = 0.02
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
def plot_boundaries(model, data_train, target_train, n_neighbors):
    data_0, data_1 = np.meshgrid(np.arange(data_train[:, 0].min() - 1, data_train[:, 0].max() + 1 , step_size),
                     np.arange(data_train[:, 1].min()-1, data_train[:, 1].max()+1, step_size))
    predictions = model.predict(np.c_[data_0.ravel(), data_1.ravel()])

    # Put the result into a color plot
    predictions = predictions.reshape(data_0.shape)
    pyplot.figure()
    pyplot.pcolormesh(data_0, data_1, predictions, cmap=cmap_light)

    # Plot also the training points
    pyplot.scatter(data_train[:, 0], data_train[:, 1], c=target_train, cmap=cmap_bold)
    pyplot.xlim(data_0.min(), data_0.max())
    pyplot.ylim(data_1.min(), data_1.max())
    pyplot.title("3-Class classification (k = %i)"
              % (n_neighbors))

    pyplot.show()

### Hyperparameters

Setting values for k

In [ ]:
K = [1, 3, 5, 7]

Plotting decision boundaries and predicting on validation split with different values for k

In [ ]:
accuracies = []

model = KNeighborsClassifier(1, weights='uniform')
model.fit(Data_train[:, :2], Target_train)

for k in K:
    model.set_params(n_neighbors=k)
    
    #Plotting boundaries
    plot_boundaries(model, Data_train, Target_train, k)
    
    #Predicting on validation split
    predictions_valid = model.predict(Data_validation[:, :2])
    score_valid = (predictions_valid[predictions_valid==Target_validation].sum())/len(Target_validation)
    print("Result for validation split with k= ",k," ",score_valid,"%")
    accuracies.append(score_valid)

Plotting accuracies for each value of k

In [ ]:
pyplot.plot(K, accuracies, 'bo')
pyplot.title('Accuracy for each k')
pyplot.xlabel('k')
pyplot.ylabel('Accuracy')

Extracting best value for k, based on validation set

In [ ]:
best_k = np.argmax(accuracies)
best_k = K[best_k]
print("Best value is ",best_k)

### Evaluating on the test set

In [ ]:
#Setting k to its best value
model.set_params(n_neighbors=best_k)

test_predictions = model.predict(Data_test[:, :2])

In [ ]:
score_test = (test_predictions[test_predictions==Target_test].sum())/len(Target_test)
print("Result for test set with k= ",best_k," ",score_test,"%")

## SVM

In [ ]:
def plot_boundaries_svm(model, data_train, target_train, c, l=-1000):
    data_0, data_1 = np.meshgrid(np.arange(data_train[:, 0].min() - 1, data_train[:, 0].max() + 1 , step_size),
                     np.arange(data_train[:, 1].min()-1, data_train[:, 1].max()+1, step_size))
    predictions = model.predict(np.c_[data_0.ravel(), data_1.ravel()])

    # Put the result into a color plot
    predictions = predictions.reshape(data_0.shape)
    pyplot.figure()
    pyplot.pcolormesh(data_0, data_1, predictions, cmap=cmap_light)

    # Plot also the training points
    pyplot.scatter(data_train[:, 0], data_train[:, 1], c=target_train, cmap=cmap_bold)
    pyplot.xlim(data_0.min(), data_0.max())
    pyplot.ylim(data_1.min(), data_1.max())
    if l == -1000:
        pyplot.title("3-Class classification (c = %i)"
              % (c))
    else:
        pyplot.title("3-Class classification (c = %i, l = %f)"
              % (c, l))

    pyplot.show()

### Hyperparameters

Setting possible values for C

In [ ]:
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [ ]:
accuracies = []
for c in C:    
    modelSVM = svm.LinearSVC(C=c)
    modelSVM.fit(Data_train[:,:2], Target_train)
    
    #Plotting boundaries
    plot_boundaries_svm(modelSVM, Data_train, Target_train, c)
    
    #Predicting on validation split
    predictions_valid = modelSVM.predict(Data_validation[:, :2])
    score_valid = (predictions_valid[predictions_valid==Target_validation].sum())/len(Target_validation)
    print("Result for validation split with c= ",c," ",score_valid,"%")
    accuracies.append(score_valid)

Plotting accuracies

In [ ]:
pyplot.plot(C, accuracies, 'bo')
pyplot.title('Accuracy for each c')
pyplot.xlabel('c')
pyplot.ylabel('Accuracy')

Extracting best value for c, based on validation set

In [ ]:
best_c = np.argmax(accuracies)
best_c = C[best_c]
print("Best value is ",best_c)

### Evaluating on the test set

In [ ]:
#Setting c to its best value
modelSVM = svm.LinearSVC(C=best_c)
modelSVM.fit(Data_train[:,:2], Target_train)

In [ ]:
test_predictions = modelSVM.predict(Data_test[:, :2])
score_test = (test_predictions[test_predictions==Target_test].sum())/len(Target_test)
print("Result for test set with c = ",best_c," ",score_test,"%")

## RBF Kernel

### Hyperparameters

In [ ]:
accuracies = []
for c in C:    
    modelSVM = svm.SVC(C=c, kernel='rbf')
    modelSVM.fit(Data_train[:,:2], Target_train)
    
    #Plotting boundaries
    plot_boundaries_svm(modelSVM, Data_train, Target_train, c)
    
    #Predicting on validation split
    predictions_valid = modelSVM.predict(Data_validation[:, :2])
    score_valid = (predictions_valid[predictions_valid==Target_validation].sum())/len(Target_validation)
    print("Result for validation split with c= ",c," ",score_valid,"%")
    accuracies.append(score_valid)

In [ ]:
best_c = np.argmax(accuracies)
best_c = C[best_c]
print("Best value is ",best_c)

### Evaluating on the test set

In [ ]:
test_predictions = modelSVM.predict(Data_test[:, :2])
score_test = (test_predictions[test_predictions==Target_test].sum())/len(Target_test)
print("Result for test set with c = ",best_c," ",score_test,"%")

### Grid search

Setting values for gamma (l)

In [ ]:
L = [0.001, 0.01, 0.1, 1, 10]

In [ ]:
accuracies = []
for c in C:
    accuracies.append([])
    for l in L:
        modelSVM = svm.SVC(C=c, kernel='rbf', gamma=l)
        modelSVM.fit(Data_train[:,:2], Target_train)
    
        #Predicting on validation split
        predictions_valid = modelSVM.predict(Data_validation[:, :2])
        score_valid = (predictions_valid[predictions_valid==Target_validation].sum())/len(Target_validation)
        print("Result for validation split with c= ",c," and l= ",l," ",score_valid,"%")
        accuracies[len(accuracies)-1].append(score_valid)

Searching for best values of c and l

In [ ]:
best_c = 0
best_l = 0
for c in range(len(C)):
    l = np.argmax(accuracies[c])
    if accuracies[c][l] > accuracies[best_c][best_l]:
        best_c = c
        best_l = l
best_c = C[best_c]
best_l = L[best_l]
print("Best value for c = ",best_c)
print("Best value for l = ",best_l)

### Evaluating on the test set

In [ ]:
modelSVM = svm.SVC(C=best_c, kernel='rbf', gamma=best_l)
modelSVM.fit(Data_train[:,:2], Target_train)
    
test_predictions = modelSVM.predict(Data_test[:, :2])
score_test = (test_predictions[test_predictions==Target_test].sum())/len(Target_test)
print("Result for test set with c = ",best_c," and l = ",best_l," ",score_test,"%")

In [ ]:
#Plotting boundaries
plot_boundaries_svm(modelSVM, Data_train, Target_train, best_c, best_l)

## K_Fold

### Merging train and validation split

In [ ]:
Data_train = np.concatenate((Data_train, Data_validation))
Target_train = np.concatenate((Target_train, Target_validation))

### Hyperparameters

Computing number of samples to take in each k-fold validation iteration

In [ ]:
n_samples = int(20/100*len(Target_train))
print(n_samples)

In [ ]:
accuracies = []
for c in C:
    accuracies.append([])
    for l in L:
        score_valids = []
        for i in range(5):
            
            #Extracting validation samples
            starting_index = i*n_samples
            Data_validation = Data_train[starting_index:starting_index+n_samples]
            Target_validation = Target_train[starting_index:starting_index+n_samples]
            
            #Extracting remaining samples
            Data_train_tmp = np.concatenate((Data_train[:starting_index], Data_train[starting_index+n_samples:]))
            Target_train_tmp = np.concatenate((Target_train[:starting_index], Target_train[starting_index+n_samples:]))
            
            #Fitting train set
            modelSVM = svm.SVC(C=c, kernel='rbf', gamma=l)
            modelSVM.fit(Data_train_tmp[:,:2], Target_train_tmp)
    
            #Predicting on validation split
            predictions_valid = modelSVM.predict(Data_validation[:, :2])
            score_valids.append((predictions_valid[predictions_valid==Target_validation].sum())/len(Target_validation))
            
        #Computing mean of result for current values of hyperparameters
        score_valid = (sum(score_valids))/5
        print("Result for validation split with c= ",c," and l= ",l," ",score_valid,"%")
        accuracies[len(accuracies)-1].append(score_valid)

Searching for best values of c and l

In [ ]:
best_c = 0
best_l = 0
for c in range(len(C)):
    l = np.argmax(accuracies[c])
    if accuracies[c][l] > accuracies[best_c][best_l]:
        best_c = c
        best_l = l
best_c = C[best_c]
best_l = L[best_l]
print("Best value for c = ",best_c)
print("Best value for l = ",best_l)

### Evaluating on the test set

In [ ]:
modelSVM = svm.SVC(C=best_c, kernel='rbf', gamma=best_l)
modelSVM.fit(Data_train[:,:2], Target_train)
    
test_predictions = modelSVM.predict(Data_test[:, :2])
score_test = (test_predictions[test_predictions==Target_test].sum())/len(Target_test)
print("Result for test set with c = ",best_c," and l = ",best_l," ",score_test,"%")